<a href="https://colab.research.google.com/github/LuanaLeite/trabalho-individual/blob/main/Entrega_1_Luana_Leite.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Trabalho Individual - Entrega 1**


> **Aluna**: Luana Leite Rodrigues 

> **Repositório**: https://github.com/LuanaLeite/TG-EDUCACAO

# **Contexto**

**A empresa**: vende produtos de emagrecimento via canais digitais. Conta com uma audiência de mais de 6M de inscritos no Youtube e, aproximadamente, 980mil seguidores no Instagram. Além da produção de conteúdo orgânico, utilização de estratégias de lançamento de produto, a empresa também investe cerca de R$800 mil reais em tráfego pago por mês para captação de leads. 

**Problemática:** a empresa está no mercado desde 2019 e, recentemente, vem enfrentando novos desafios em relação ao custo de aquisição de clientes e redução de margem de lucro. Dado esse cenário, a grande pergunta feita pelo principal gestor da empresa, demandante da análise, foi "*Quem são os nossos melhores clientes? O que eles compraram?*".   



# **Dados fornecidos**

Para responder a pergunta feita pelo gestor da empresa, foi disponibilizada uma base de vendas histórica de janeiro de 2020 até novembro de 2022. A base é constituída do histórico de 5 plataformas de vendas (hotmart, guru-info, guru-sup, upnid e nutlog). Para organização e manipulação dos dados, foi concatenado em excel, com trativas específicas para deixar as bases no mesmo formato e padronizar seus campos. 

Para melhor compreensão da base extraída para análise, confira o **dicionário de variáveis**: [dicionário de variáveis.](https://docs.google.com/spreadsheets/d/1h7uDHroNP6_lTfM1Is5HvlHgBBAP84sNmIXJD1hWgEo/edit?usp=sharing)

## *Coleta e pré-processamento*



In [1]:
# Common standard libraries

import datetime
import time
import os

In [ ]:
# Common external libraries

import pandas as pd
import numpy as np
import sklearn # scikit-learn
import requests
from bs4 import BeautifulSoup
import math 

In [ ]:
# Visualization libraries

%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
# Setting plot appearance

%config InlineBackend.figure_format='retina'
sns.set() # Revert to matplotlib defaults
plt.rcParams['figure.figsize'] = (9, 6)
plt.rcParams['axes.labelpad'] = 10
sns.set_style("darkgrid")
# sns.set_context("poster", font_scale=1.0)

Importação dos dados

In [ ]:

from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
df = pd.read_csv('/content/drive/My Drive/vita_dados.csv', sep=';')

In [ ]:
df.shape

(224978, 47)

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 224978 entries, 0 to 224977
Data columns (total 47 columns):
 #   Column                  Non-Null Count   Dtype  
---  ------                  --------------   -----  
 0   base dados              224978 non-null  object 
 1   idTransação             224974 non-null  object 
 2   marketplace             224978 non-null  object 
 3   status                  224978 non-null  object 
 4   pagamento               224978 non-null  object 
 5   parcelas                224978 non-null  int64  
 6   moeda                   224978 non-null  object 
 7   valorVenda              224978 non-null  float64
 8   valorMarketplace        224978 non-null  float64
 9   valorLiquido            224978 non-null  float64
 10  valorProduto            223199 non-null  float64
 11  valorParcela            125580 non-null  float64
 12  idProduto               213903 non-null  float64
 13  nomeProduto             224978 non-null  object 
 14  qtdProduto          

In [ ]:
df = df.astype({"dataPedido":'datetime64[ms]',
                "dataAprovacao":'datetime64[ms]',
                "dataCancelamento":'datetime64[ms]',
                "dataGarantia":'datetime64[ms]',
                "vencimento_boleto":'datetime64[ms]',
                "DataPrimeiraCompra":'datetime64[ms]'})



In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 224978 entries, 0 to 224977
Data columns (total 47 columns):
 #   Column                  Non-Null Count   Dtype         
---  ------                  --------------   -----         
 0   base dados              224978 non-null  object        
 1   idTransação             224974 non-null  object        
 2   marketplace             224978 non-null  object        
 3   status                  224978 non-null  object        
 4   pagamento               224978 non-null  object        
 5   parcelas                224978 non-null  int64         
 6   moeda                   224978 non-null  object        
 7   valorVenda              224978 non-null  float64       
 8   valorMarketplace        224978 non-null  float64       
 9   valorLiquido            224978 non-null  float64       
 10  valorProduto            223199 non-null  float64       
 11  valorParcela            125580 non-null  float64       
 12  idProduto               213903

In [ ]:
print(df)

       base dados                           idTransação marketplace  \
0           upnid                               1418756       upnid   
1         hotmart                      HP12915778937701     hotmart   
2         hotmart                      HP17315779087437     hotmart   
3         hotmart                      HP14315779180064     hotmart   
4         hotmart                      HP05815779272122     hotmart   
...           ...                                   ...         ...   
224973  guru_supl  97df5799-e95f-4684-9c7f-143dadd07e6b     PagarMe   
224974  guru_supl  97df5ba5-78d3-4538-91d9-5607e59058f2     PagarMe   
224975  guru_supl  97df6296-bab2-46c1-b1e1-c237d6c98cc0     PagarMe   
224976  guru_supl  97df65eb-3c34-4d17-bef6-0251ea7c45bb     PagarMe   
224977  guru_supl  97df770b-ac0d-420c-a278-43d6b5d88179     PagarMe   

            status          pagamento  parcelas moeda  valorVenda  \
0       Chargeback  Cartão de Crédito        12   BRL      447.00   
1        

In [ ]:
df.head()

,base dados,idTransação,marketplace,status,pagamento,parcelas,moeda,valorVenda,valorMarketplace,valorLiquido,...,utm_medium,utm_content,vencimento_boleto,oferta,assinaturaCódigo,assinaturaCiclo,PrimeiroProduto,DataPrimeiraCompra,#Transação,ClassificaçãoTransação
0,upnid,1418756,upnid,Chargeback,Cartão de Crédito,12,BRL,447.0,23.35,423.65,...,NaN,NaN,NaT,NaN,NaN,0,Fórmula SB3 - Kit com 3 Potes (35% Off),2019-11-28,1,Primeira Compra
1,hotmart,HP12915778937701,hotmart,Completo,Cartão de Crédito,1,BRL,99.0,8.82,90.18,...,NaN,NaN,NaT,kytlmklm,NaN,0,Ebook Detox de 15 Dias,2020-01-01,1,Primeira Compra
2,hotmart,HP17315779087437,hotmart,Completo,Cartão de Crédito,1,BRL,97.0,8.66,88.34,...,NaN,NaN,NaT,2opgc0eg,NaN,0,Kit Dayan Siebra,2020-01-01,1,Primeira Compra
3,hotmart,HP14315779180064,hotmart,Completo,Cartão de Crédito,1,BRL,99.0,8.82,90.18,...,NaN,NaN,NaT,kytlmklm,NaN,0,Ebook Detox de 15 Dias,2020-01-01,1,Primeira Compra
4,hotmart,HP05815779272122,hotmart,Completo,Boleto Bancário,1,BRL,99.0,8.82,90.18,...,NaN,NaN,NaT,kytlmklm,NaN,0,Ebook Detox de 15 Dias,2020-01-01,1,Primeira Compra


In [ ]:
df[['valorLiquido']].describe()

,valorLiquido
count,224978.000000
mean,371.410874
std,328.853420
min,-0.070000
25%,143.200000
50%,206.000000
75%,538.780000
max,2498.840000


In [ ]:
df[['marketplace']].describe()

,marketplace
count,224978
unique,6
top,PagarMe
freq,124938


In [ ]:
df.describe().T

,count,mean,std,min,25%,50%,75%,max
parcelas,224978.0,6.747802e+00,4.983919e+00,1.00,1.00,6.00,12.00,1.200000e+01
valorVenda,224978.0,3.911672e+02,3.465348e+02,0.00,153.90,209.00,577.32,2.499240e+03
valorMarketplace,224978.0,3.496703e+01,5.926791e+01,0.00,3.56,16.48,36.68,8.334800e+02
valorLiquido,224978.0,3.714109e+02,3.288534e+02,-0.07,143.20,206.00,538.78,2.498840e+03
valorProduto,223199.0,3.756213e+02,1.845552e+03,0.00,153.90,206.00,499.00,8.559870e+05
valorParcela,125580.0,1.713461e+02,2.387557e+02,0.49,23.61,85.71,180.59,2.082000e+03
idProduto,213903.0,2.541295e+08,5.943162e+08,1.00,12.00,406542.00,652159.00,1.667914e+09
qtdProduto,224978.0,1.000000e+00,0.000000e+00,1.00,1.00,1.00,1.00,1.000000e+00
assinaturaCiclo,224978.0,6.672741e-01,1.242673e+00,0.00,0.00,0.00,1.00,2.900000e+01
#Transação,224978.0,2.094098e+00,2.440199e+00,1.00,1.00,1.00,2.00,7.600000e+01
